In [1]:
from transformers import WhisperProcessor
import datasets
from datasets import Audio

/home/mihailo/miniconda3/envs/transformers/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def transliterate_cir2lat(text: str) -> str:
    """
    Pretvara tekst napisan ćirilicom u latinicu
    :param text: Tekst na ćirilici
    :return: Tekst na latinici
    """
    mappings = {
        "а": "a",
        "б": "b",
        "в": "v",
        "г": "g",
        "д": "d",
        "ђ": "đ",
        "е": "e",
        "ж": "ž",
        "з": "z",
        "и": "i",
        "ј": "j",
        "к": "k",
        "л": "l",
        "љ": "lj",
        "м": "m",
        "н": "n",
        "њ": "nj",
        "о": "o",
        "п": "p",
        "р": "r",
        "с": "s",
        "т": "t",
        "ћ": "ć",
        "у": "u",
        "ф": "f",
        "х": "h",
        "ц": "c",
        "ч": "č",
        "џ": "dž",
        "ш": "š",
        "А": "A",
        "Б": "B",
        "В": "V",
        "Г": "G",
        "Д": "D",
        "Ђ": "Đ",
        "Е": "E",
        "Ж": "Ž",
        "З": "Z",
        "И": "I",
        "Ј": "J",
        "К": "K",
        "Л": "L",
        "Љ": "Lj",
        "М": "M",
        "Н": "N",
        "Њ": "Nj",
        "О": "O",
        "П": "P",
        "Р": "R",
        "С": "S",
        "Т": "T",
        "Ћ": "Ć",
        "У": "U",
        "Ф": "F",
        "Х": "H",
        "Ц": "C",
        "Ч": "Č",
        "Џ": "Dž",
        "Ш": "Š",
    }
    translit = ""
    for char in text:
        if char in mappings.keys():
            translit = translit + mappings[char]
        else:
            translit = translit + char
    return translit

In [3]:
# include serbian, bosnian and croatian
dataset_configs_to_use = [
    {
        "dataset_name": "mozilla-foundation/common_voice_13_0",
        "languages": ["sr", "bs", "hr"],
        "audio_column": "audio",
        "text_column": "sentence",
    },
    {
        "dataset_name": "facebook/voxpopuli",
        "languages": ["hr"],
        "audio_column": "audio",
        "text_column": "sentence",
    },
    {
        "dataset_name": "google/fleurs",
        "languages": ["Serbian", "Croatian", "Bosnian"],
        "audio_column": "audio",
        "text_column": "sentence",
    },
]
processor = WhisperProcessor.from_pretrained(
    "openai/whisper-small", language="sinhalese", task="transcribe"
)

In [4]:
def preprocess_data(sample):
    sample["text"] = transliterate_cir2lat(sample["text"])
    return sample


def create_dataset():
    dataset_list = []
    for config in dataset_configs_to_use:
        for language in config["languages"]:
            print("DATASETNAME:", config["dataset_name"])
            data = datasets.load_dataset(
                config["dataset_name"], language, split="train+validation+test"
            )
            data = datasets.Dataset.from_dict(
                {
                    "text": data[config["text_column"]],
                    "audio": data[config["audio_column"]],
                }
            )
            sampling_rate = processor.feature_extractor.sampling_rate()
            data = data.cast_column("audio", Audio(sampling_rate=sampling_rate))
            new_data = data.map(
                preprocess_data,
                input_columns=[config["audio_column"], config["text_column"]],
            )
            dataset_list.append(new_data)

    new_dataset = datasets.concatenate_datasets(dataset_list)
    new_dataset = new_dataset.shuffle(seed=42)
    return new_dataset


dataset = create_dataset()

DATASETNAME: mozilla-foundation/common_voice_13_0


/home/mihailo/miniconda3/envs/transformers/lib/python3.10/site-packages/datasets/load.py:1429: FutureWarning: The repository for mozilla-foundation/common_voice_13_0 contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mozilla-foundation/common_voice_13_0
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


: 

In [ ]:
dataset.save_to_disk("test_data")